Deep One-Class Classification
http://proceedings.mlr.press/v80/ruff18a/ruff18a.pdf

Deep Semi-Supervised Anoamly Detection
https://arxiv.org/pdf/1906.02694.pdf


(Supervised) Contrastive Loss

CSI: Novelty Detection via Contrastive Learningon Distributionally Shifted Instances : https://arxiv.org/pdf/2007.08176.pdf

A Unifying Review of Deep and Shallow AnomalyDetection : https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9347460&tag=1

Dropout techniques on RNNs

https://adriangcoder.medium.com/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b


In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import h5py
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas as pd
from tqdm.autonotebook import tqdm
from sklearn.manifold import TSNE
# from tsne_torch import TorchTSNE as TSNE
from sklearn.cluster import KMeans
from math import ceil
from glob import glob
from copy import deepcopy
from collections import deque
from sklearn.preprocessing import MinMaxScaler
import sys
sys.path.insert(0, '../code/')
sys.path.insert(0, './models/')
import util
from models import *
import dataset
import os
import pickle
import re
import math
from glob import glob
from pathlib import Path

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

t2n = lambda x : x.cpu().detach().numpy()


titles = ['PROBE.AMPL.TD', 'PROBE.PHASE.TD', \
          'VFORW.AMPL.TD', 'VFORW.PHASE.TD', \
          'VREFL.AMPL.TD', 'VREFL.PHASE.TD']

parameters = {
    'input_dims' : [0,1,2,3,4,5],
    # 'input_dims' : [0,2,4],
    'num_epochs' : 128,
    'learning_rate' : 0.001,
    'weight_decay' : 10e-4,
    'eta_anom' : 1.0,
    'eta_label' : 5.0,
    'eta_network' : 0, # 1/2
    'n_batches' : 32,
    'test_batches': 3,
    'num_hidden' : 256,
    'num_hidden_layers' : 1}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#
#
# --- NETWORK & OPTIM ---
#

'''
# regular
model = QuenchDetectionNetworkLSTM(parameters['num_hidden'],
                                   parameters['num_hidden_layers'],
                                   parameters['eta_anom'], 
                                   parameters['eta_label'], 
                                   parameters['eta_network'],
                                   len(parameters['input_dims'])).to(device)
model.c = model.c.to(device)
folder = '../models/LSTM_256_LSTM1_Linear1/eta_anom_1_eta_label_5_feb22_2/'

'''
# linear beteween
'''
model = QuenchDetectionNetworkLSTM2LinearBetween(num_hidden = parameters['num_hidden'],
                                   num_hidden_layers = parameters['num_hidden_layers'],
                                   eta_anom = parameters['eta_anom'], 
                                   eta_label = parameters['eta_label'], 
                                   eta_network = parameters['eta_network']).to(device)
folder = '../models/LSTM_256_LSTM2_Linear1_LinearBetween/eta_anom_1_eta_label_5/'
'''

# layer norm.  
'''
model = QuenchDetectionNetworkLSTM2LayerNorm(num_hidden = parameters['num_hidden'],
                                   num_hidden_layers = parameters['num_hidden_layers'],
                                   eta_anom = parameters['eta_anom'], 
                                   eta_label = parameters['eta_label'], 
                                   eta_network = parameters['eta_network']).to(device)
folder = '../models/LSTM_256_LSTM2_Linear1_LayerNorm/eta_anom_1_eta_label_5/'
'''

# softmax loss - i.e. a regular classifier
'''
model = QuenchDetectionNetworkClassifierLSTM(num_hidden = parameters['num_hidden'],
                                   num_hidden_layers = parameters['num_hidden_layers'],
                                   eta_anom = parameters['eta_anom'], 
                                   eta_label = parameters['eta_label'], 
                                   eta_network = parameters['eta_network']).to(device)
folder = '../models/LSTM_256_LSTM1_Linear1_Classifier/eta_anom_1_eta_label_5_2(same as OC)_2/'
'''

if not os.path.exists(folder):
    print(f'creating {folder}')
    os.makedirs(folder)

optimizer = torch.optim.Adam(model.parameters(),lr=parameters['learning_rate'], weight_decay = parameters['weight_decay'])
print(model)
last_epoch = 0

weights_file = sorted(glob(folder + 'anomaly_epoch_0*'))[-1]
print(f'last epoch {last_epoch} from file {weights_file}')
last_epoch = int(weights_file.split('_')[-1])
model.load_state_dict(torch.load(weights_file))

QuenchDetectionNetworkLSTM(
  (input_rnn): LSTM(1092, 256, bias=False)
  (hidden_rnn): LSTM(256, 256, bias=False)
  (hidden_lin): Linear(in_features=256, out_features=256, bias=False)
  (dropout): Dropout(p=0.0, inplace=False)
)
last epoch 0 from file ../models/LSTM_256_LSTM1_Linear1/eta_anom_1_eta_label_5_feb22_nobias_const_c/anomaly_epoch_0019


RuntimeError: Error(s) in loading state_dict for QuenchDetectionNetworkLSTM:
	Missing key(s) in state_dict: "input_rnn.weight_ih_l0", "input_rnn.weight_hh_l0", "hidden_rnn.weight_ih_l0", "hidden_rnn.weight_hh_l0", "hidden_lin.weight". 
	Unexpected key(s) in state_dict: "cnn.0.weight", "cnn.2.weight", "cnn.4.weight", "linear.weight". 

# Loading Data

In [2]:
faulty_folders = ['/home/sulcan/data/quench/down_labelled/']
faulty_files = ['/home/sulcan/data/quench/X_down_labelled_all.pickle']
# faulty_files = ['/home/sulcan/data/quench/X_down_labelled_classified_all.pickle']

healthly_files =   ['/home/sulcan/data/quench/X_up_2021-08_all.pickle',
                    '/home/sulcan/data/quench/X_up_2021-10_all.pickle', 
                    '/home/sulcan/data/quench/X_up_2021-11_all.pickle',
                    '/home/sulcan/data/quench/X_up_2022-01_all.pickle',
                    '/home/sulcan/data/quench/X_up_2022-02_all.pickle'
                   ]

healthly_folders = [None] * len(healthly_files) # using [None] causes an exception in case of an error in path and prevent the mundane processing the existing files

X_faulty, files_and_locations_faulty, pids_faulty = dataset.load_multiple_data_from_numpy(faulty_files, faulty_folders, parameters['input_dims'])
X_healthly, files_and_locations_healthly, pids_healthly = dataset.load_multiple_data_from_numpy(healthly_files, healthly_folders, parameters['input_dims'])

file /home/sulcan/data/quench/X_down_labelled_all.pickle exists, loading file
file /home/sulcan/data/quench/X_down_labelled_all.pickle loaded
file /home/sulcan/data/quench/X_up_2021-08_all.pickle exists, loading file
file /home/sulcan/data/quench/X_up_2021-08_all.pickle loaded
file /home/sulcan/data/quench/X_up_2021-10_all.pickle exists, loading file
file /home/sulcan/data/quench/X_up_2021-10_all.pickle loaded
file /home/sulcan/data/quench/X_up_2021-11_all.pickle exists, loading file
file /home/sulcan/data/quench/X_up_2021-11_all.pickle loaded
file /home/sulcan/data/quench/X_up_2022-01_all.pickle exists, loading file
file /home/sulcan/data/quench/X_up_2022-01_all.pickle loaded
file /home/sulcan/data/quench/X_up_2022-02_all.pickle exists, loading file
file /home/sulcan/data/quench/X_up_2022-02_all.pickle loaded


In [3]:
crit = lambda x : x.shape[-1] > 0
print(f'Filtering out empty healthly events {len(X_healthly)}')
X_healthly = list(filter(crit, X_healthly))
print(f'Remaining healthly events {len(X_healthly)}')

print(f'Filtering out empty faulty events {len(X_faulty)}')
X_faulty = list(filter(crit, X_faulty))
print(f'Remaining faulty events {len(X_faulty)}')

Filtering out empty healthly events 81922
Remaining healthly events 81922
Filtering out empty faulty events 1331
Remaining faulty events 1331


In [4]:
# --- putting everything together into an input----
input = X_healthly + X_faulty

# (A,P) -> (I,Q)
if len(parameters['input_dims']) == 6: 
    input = dataset.transform_RF_pulses(input)
# just selecting subset of values
elif len(parameters['input_dims']) == 3:
    input = dataset.select_dimensions(input,parameters['input_dims'])
input = dataset.normalize_data(input)

# --- creating labels ----
labels = [+1] * len(X_healthly) + [-1] * len(X_faulty)

print(f'healthly {len(X_healthly)} faulty {len(X_faulty)}')

del X_faulty
del X_healthly

  0%|          | 0/83253 [00:00<?, ?it/s]

  0%|          | 0/83253 [00:00<?, ?it/s]

healthly 81922 faulty 1331


In [5]:
# permutation for batch indices
if os.path.isfile(folder + '/permutation.npy'):
    print('loading permutation')
    permutation = np.load(folder + '/permutation.npy', allow_pickle = True)
else:
    print('creating permutation & saving it')
    permutation = dataset.generate_permutation_for_batch_indices(input, parameters['n_batches'])
    np.save(folder + '/permutation.npy',permutation)

with open(folder + 'training_data.pickle','wb') as f:
    pickle.dump({'healthly_files' : healthly_files, \
                 'faulty_files' : faulty_files, \
                 'files_and_locations_healthly' : files_and_locations_healthly, 
                 'files_and_locations_faulty' : files_and_locations_faulty}, f)

# we should make sure that the first batch has a certain number of faults
print('number of faults in first batch', np.sum([labels[i] == -1 for i in permutation[0]]))

assert len(set([i for p in permutation for i in p])) == len(input)
assert len(input) == len(labels)

loading permutation
number of faults in first batch 33


# Training

In [7]:
''' TODO '''

class QuenchDetectionNetworkCNN(nn.Module):
    def __init__(self):
        super(QuenchDetectionNetworkCNN,self).__init__()
        self.cnn = nn.Sequential(nn.Conv2d(6,8,8,stride = 1, bias = False),
                              nn.ReLU(),
                              nn.Conv2d(8,16, 8,stride = 1, bias = False),
                              nn.ReLU(),
                              nn.Conv2d(16,32,8,stride = 1, bias = False))
        self.linear = nn.Linear(320, 16, bias = False)
        self.c = torch.randn(1,16)    
    
    def phi(self, x):
        
        # filtering out too short sequences
        lengths = torch.tensor([x_.shape[0] for x_ in x])
        x = [x[i] for i in torch.where(lengths > 64)[0]]
        # the calculation itself, cnn first
        x = [x.reshape((-1,6,182)).transpose(1,0).unsqueeze(0) for x in x]
        # then summing out all sequences, different kernels should also be tried (maybe max?)
        x = torch.cat([self.cnn(x).sum(dim = 2).reshape((1,-1)) for x in x])
        print(x[0].shape)
        # linear output
        return self.linear(x)
    
    def forward(self, x):
        x = self.phi(x)
        return torch.linalg.vector_norm(x - self.c,dim = -1), x

    def loss(self, D,L):
        s, _ = self.forward(D)
        print(s ** L)
        return None
    
model = QuenchDetectionNetworkCNN().to(device)
model.c = model.c.to(device)
# model.loss(D,L)

In [11]:
print(len(D))

2601


In [10]:

with torch.no_grad():
    res = model(D)

torch.Size([5152])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2574x5152 and 320x16)

In [9]:
losses = []
losses_R = []
losses_D = []
losses_W = []

for epoch in range(last_epoch+1, parameters['num_epochs']):
    for batch in range(parameters['test_batches'],parameters['n_batches']):
        indices = permutation[batch]
        N = len(indices)
        
        # input data
        D = [torch.tensor(input[i]).to(device) for i in indices]
        # L == -1 # (-1) quench
        # L == +1 # (+1) proper
        L = torch.stack([torch.tensor(labels[i]).to(device) for i in indices]).float().flatten()
        break
        # ===================forward=====================        
        loss_R, loss_D, loss_W = model.loss(D,L)
        loss = loss_R + loss_D + loss_W
        
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'LOSS (epoch {epoch},batch {batch}) {loss.item()} = R({loss_R.item()}) + D({loss_D.item()}) + W({loss_W.item()})')
        
        losses.append(loss.item())
        losses_R.append(loss_R.item())
        losses_D.append(loss_D.item())
        losses_W.append(loss_W.item())
    break
    plt.plot(losses,'b')
    plt.plot(losses_R,'g')
    plt.plot(losses_D,'r')
    plt.plot(losses_W,'c')
    plt.show()
    torch.save(model.state_dict(),folder + '/anomaly_epoch_' + str(epoch).zfill(4))